# Visualizing intermediate activation in Convolutional Neural Networks with Keras

* Medium post it's based on: https://towardsdatascience.com/visualizing-intermediate-activation-in-convolutional-neural-networks-with-keras-260b36d60d0 
* GitHub version with full code: https://github.com/gabrielpierobon/cnnshapes/blob/master/README.md


> "In order to extract the feature maps we want to look at, we’ll create a Keras model that takes batches of images as input, and outputs the activations of all convolution and pooling layers. To do this, we’ll use the Keras class Model. A model is instantiated using two arguments: an input tensor (or list of input tensors) and an output tensor (or list of output tensors). The resulting class is a Keras model, just like the Sequential models, mapping the specified inputs to the specified outputs. What sets the Model class apart is that it allows for models with multiple outputs, unlike Sequential."

## Code for printing all layers 

Once you have trained a CNN model, change the model name in the code below and run the code. It should print out the images from each layer.

Note that I'm having a little trouble with showing all the images. It so far only shows multiples of `images_per_row` which I've set to 6. So it shows all 6 images in the first two layers of LeNet5 and the first 12 images of the next two layers that actually have 16 images each.

I'll fix it to it runs on VGG--it currently runs out of memory. 

Scroll down for code that shows the images of a single layer that you choose. 


In [0]:
# Code Modified from: https://towardsdatascience.com/visualizing-intermediate-activation-in-convolutional-neural-networks-with-keras-260b36d60d0

# Model to feed in
model = LeNet5_model2

# Index of the sample image 
imgnum=3

# Print out all the layers, their names, and shapes 
print(model.summary())

# These are the layer outputs 
layer_outputs = [layer.output for layer in model.layers]

activation_model = models.Model(inputs=model.input, outputs=layer_outputs)

# This is where you predict an input. I'm gonna try predicting just one image later, instead of the whole dev set, since VGG 16 is running out of memory 
activations = activation_model.predict(X_dev) 
# Each element in activations is a separate layer. Refer to summary for what these look like. 
# You could also do something like: activation[0].shape for the shape of the first layer 
# The shape will be in this form:   number of examples X size of feature map x size of feature map x number of channels

layer_names = []
for layer in model.layers:
    layer_names.append(layer.name) # Names of the layers, so you can have them as part of your plot
    
images_per_row = 6    

for layer_name, layer_activation in zip(layer_names, activations): # Displays the feature maps
    ## Only if shape has 4 numbers. So ignore flattened/fully connected layers
    if len(layer_activation.shape) == 4:
      
      n_features = layer_activation.shape[-1] # Number of features in the feature map
      size = layer_activation.shape[1] #The feature map has shape (1, size, size, n_features).

      
      n_cols =  n_features // images_per_row # Tiles the activation channels in this matrix
      display_grid = np.zeros((size * n_cols, images_per_row * size))

      for col in range(n_cols): # Tiles each filter into a big horizontal grid
          for row in range(images_per_row):
              channel_image = layer_activation[imgnum,   # image number
                                                :, :,
                                                col * images_per_row + row]
              # Post-processes the feature to make it visually palatable
              channel_image -= channel_image.mean() 
              channel_image /= channel_image.std()
              channel_image *= 64
              channel_image += 128
              channel_image = np.clip(channel_image, 0, 255).astype('uint8')
              display_grid[col * size : (col + 1) * size, # Displays the grid
                            row * size : (row + 1) * size] = channel_image
      scale = 1. / size
      plt.figure(figsize=(scale * max(display_grid.shape[1], 1),
                          scale * max(display_grid.shape[0], 1)))
      plt.title(layer_name)
      plt.grid(False)
      plt.imshow(display_grid, aspect='auto', cmap="gray")

## Printing a single channel in a single layer

The code above is for looping through all channels in all layers. The code below is to visualize select channels in a single layer.

In [0]:
# I want to see every layer, actually.
# Let's stick to image indexed 0

def show_img_n_1stlayer(img_index, channel_index, layer_activation):
  fig, axis = plt.subplots(1, 2)
  fig.suptitle("Layer index:" + str(channel_index))
  
  # First image is the original face
  axis[0].imshow(X_dev[img_index][:, :, 0],cmap="gray")
  axis[0].axis("off")

  # Second image is the nth filter of the first layer 
  axis[1].imshow(layer_activation[img_index, :, :, channel_index],cmap="gray")
  axis[1].axis("off")

  fig.show()


# Choose layer here
layer_index = 1
# Choose channel here
channel_num = 0
# sample image
imgnum=3


show_img_n_1stlayer(img_index = imgnum, 
                    channel_index=channel_num, 
                    layer_activation=activations[layer_index])